## ndarrayとshowataの使い方

In [2]:
:dep showata = { version = "0.3.2", features=["show_ndarray"]}
:dep ndarray = "0.14"
use showata::Showable;

### ベクトルの作成と表示

In [3]:
use ndarray::Array1;
let m: Array1<f64> = Array1::<f64>::zeros(3);
m.show()

,0,1,2
0,0.0,0.0,0.0


### 行列の作成と表示

In [4]:
use ndarray::Array2;

let data = Array2::<f64>::zeros((3, 4));
data.show().unwrap();

,0,1,2,3
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0


In [5]:
use ndarray::Array;
let arr = Array::from_shape_vec((2, 3), vec![0, 1, 2, 3, 4, 5]).unwrap();
arr

,0,1,2
0,0,1,2
1,3,4,5


## polarsの使い方

In [6]:
:dep polars = { version = "0.18.0", features = ["ndarray", "random"]}

In [7]:
use polars::prelude::*;

### シリーズの作成

In [8]:
let t: Series = [1, 2, 3].iter().collect();
t

The type of the variable arr was redefined, so was lost.
The type of the variable m was redefined, so was lost.
The type of the variable data was redefined, so was lost.


shape: (3,)
Series: '' [i32]
[
	1
	2
	3
]

### データフレームの作成 

In [9]:
let mut df = df!("A" => &["a", "b", "a", "c"],
             "B" => &[1, 3, 5, 7],
             "C" => &[10, 11, 12, 13],
             "D" => &[2, 4, 6, 8]
    )?;
df

shape: (4, 4)
+-----+-----+-----+-----+
| A   | B   | C   | D   |
| --- | --- | --- | --- |
| str | i32 | i32 | i32 |
+=====+=====+=====+=====+
| a   | 1   | 10  | 2   |
+-----+-----+-----+-----+
| b   | 3   | 11  | 4   |
+-----+-----+-----+-----+
| a   | 5   | 12  | 6   |
+-----+-----+-----+-----+
| c   | 7   | 13  | 8   |
+-----+-----+-----+-----+


In [10]:
// 列選択(A列、B列を選択)
let selected = df.select(("A", "B"))?;
selected

shape: (4, 2)
+-----+-----+
| A   | B   |
| --- | --- |
| str | i32 |
+=====+=====+
| a   | 1   |
+-----+-----+
| b   | 3   |
+-----+-----+
| a   | 5   |
+-----+-----+
| c   | 7   |
+-----+-----+


In [11]:
// 行抽出(B列が4以下の行を抽出)
let selected2 = df.filter(&df.column("B")?.lt_eq(4))?;
selected2

shape: (2, 4)
+-----+-----+-----+-----+
| A   | B   | C   | D   |
| --- | --- | --- | --- |
| str | i32 | i32 | i32 |
+=====+=====+=====+=====+
| a   | 1   | 10  | 2   |
+-----+-----+-----+-----+
| b   | 3   | 11  | 4   |
+-----+-----+-----+-----+


## plottersの使い方

In [12]:
:dep plotters = { git = "https://github.com/38/plotters", default_features = false, features = ["evcxr", "all_series", "all_elements"]}
extern crate plotters;
use plotters::prelude::*;

### ヒストグラムの描画

In [13]:
:dep rand = { version = "0.6.5" }
extern crate rand;

use rand::distributions::Normal;
use rand::distributions::Distribution;
use rand::thread_rng;
let sd = 0.13;
let random_points:Vec<(f64,f64)> = {
    let mut norm_dist = Normal::new(0.5, sd);
    let (mut x_rand, mut y_rand) = (thread_rng(), thread_rng());
    let x_iter = norm_dist.sample_iter(&mut x_rand);
    let y_iter = norm_dist.sample_iter(&mut y_rand);
    x_iter.zip(y_iter).take(1000).collect()
};

The type of the variable selected2 was redefined, so was lost.
The type of the variable t was redefined, so was lost.
The type of the variable df was redefined, so was lost.
The type of the variable selected was redefined, so was lost.


In [14]:
evcxr_figure((640, 480), |root| {
    let areas = root.split_evenly((2,1));
    let mut charts = vec![];
    
    // The following code will create a chart context
   for (area, name) in areas.iter().zip(["X", "Y"].into_iter()) {
        let mut chart = ChartBuilder::on(&area)
            .caption(format!("Histogram for {}", name), ("Arial", 20).into_font())
            .x_label_area_size(40)
            .y_label_area_size(40)
            .build_cartesian_2d(0u32..100u32, 0f64..0.5f64)?;
        chart.configure_mesh()
            .disable_x_mesh()
            .disable_y_mesh()
            .y_labels(5)
            .x_label_formatter(&|x| format!("{:.1}", *x as f64 / 100.0))
            .y_label_formatter(&|y| format!("{}%", (*y * 100.0) as u32))
            .draw()?;
        charts.push(chart);
    }
    
    let hist_x = Histogram::vertical(&charts[0])
        .style(RED.filled())
        .margin(0)
        .data(random_points.iter().map(|(x,_)| ((x*100.0) as u32, 0.01)));
    
    let hist_y = Histogram::vertical(&charts[0])
        .style(GREEN.filled())
        .margin(0)
        .data(random_points.iter().map(|(_,y)| ((y*100.0) as u32, 0.01)));
    
    charts[0].draw_series(hist_x);
    charts[1].draw_series(hist_y);
    
    Ok(())
}).style("width:60%")

Histogram for X
 
 
 
10%
 
 
 
20%
 
 
 
30%
 
 
 
40%
 
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
Histogram for Y
 
 
 
10%
 
 
 
20%
 
 
 
30%
 
 
 
40%
 
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0

### 散布図の描画

In [15]:
evcxr_figure((640, 480), |root| {
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
        .caption("Normal Distribution", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_ranged(0f64..1f64, 0f64..1f64)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .draw()?;
    
    chart.draw_series(random_points.iter().map(|(x,y)| Circle::new((*x,*y), 3, GREEN.filled())));
    
    // You can alawys freely draw on the drawing backend
    let area = chart.plotting_area();
    let two_sigma = sd * 2.0;
    area.draw(&Rectangle::new(
        [(0.5 - two_sigma, 0.5 - two_sigma), (0.5 + two_sigma, 0.5 + two_sigma)], 
        RED.mix(0.3).filled())
    )?;
    area.draw(&Cross::new((0.5, 0.5), 5, &RED))?;
    
    Ok(())
}).style("width:60%")

Normal Distribution
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0

In [16]:
:vars

Variable,Type
random_points,"Vec<(f64,f64)>"
sd,f64


In [2]:
:timing // 時間測定をオンにする（トグル式）
use std::thread::sleep;
use std::time::Duration;
sleep(Duration::from_millis(2000));

Timing: true


Took 2378ms

# 参考
- [evcxr_jupyter - GitHub](https://github.com/google/evcxr/blob/main/evcxr_jupyter)
- [showata -GitHub](https://github.com/procyon-rs/showata)
- [ndarray - GitHub](https://github.com/rust-ndarray/ndarray)
- [polars -GitHub](https://github.com/pola-rs/polars)
- [plotters -GitHub](https://github.com/38/plotters)
